In [35]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from pytrends import dailydata
import requests
import collections

from sklearn import linear_model
import statsmodels.api as sm

In [36]:
pd.set_option('display.max_columns', None)

In [37]:
with open("words.txt") as f:
    words = f.readlines()
words = [x.strip() for x in words]
words = words[:5] + ["debt"]
words

['hedge', 'dividend', 'earnings', 'inflation', 'markets', 'debt']

In [73]:
length = 103
metric = 0

profits = {} # keeps track of words with best profit
word_types = collections.defaultdict(list) # keeps track of which words have positive vs negative sentiment

In [83]:
def get_trend(key_word, start_year, end_year, start_month, end_month, profits, word_types):
    trend = pd.DataFrame(dailydata.get_daily_data(key_word, start_year, start_month, end_year, end_month, wait_time = 0)[key_word])
    trend["change"] = trend.pct_change()
    
    # grab DJIA data
    djia = yf.download("DJIA", start = str(start_year)+"-"+str(start_month)+"-01", end = str(end_year)+"-"+str(end_month)+"-01")["Adj Close"]
    
    # join trend data with DJIA data
    joined = trend.merge(djia, left_on = trend.index, right_on = djia.index)
    joined = joined.rename(columns = {"key_0": "Date"})
    
    # grab the adj close price difference for each day
    joined['Diff'] = joined['Adj Close'].diff()
    
    
    # we implement the algorithm described, where if the change is greater than some metric,
    # then we sell at the current price (assume we own at start) and buy the following week at that price,
    # this then corresponds to subtracting the difference between week i+1 and week i (we hope price decreased, leading to positive gain)
    # we do the opposite for the less than case
    # NOTE: assumes that these words have negative sentiment, ie that increased search volume will lead to a decrease in price
    
    def getVal(row):
        if row.name == 0:
            return 0
        elif row.name < length-1 and joined['change'].iloc[row.name+1] > metric:
            return joined['Diff'].iloc[row.name+1]
        elif row.name < length-1 and joined['change'].iloc[row.name+1] < -metric:
            return joined['Diff'].iloc[row.name+1]
        else:
            return 0
        
    # determine whether to add or subtract the difference
    # if the % change is > 0 (or some arbitrary metric), then we know to sell the asset and rebuy at the next week
    # if % change is < 0, then we buy the asset and resell it the next week
    joined['Recent Profit'] = joined.apply(getVal, axis = 1)
    
    # get cumulative returns
    joined['Total Profit'] = joined['Recent Profit'].cumsum()
    
    # record final profit (may be slightly off due to buying/selling at beginning/end)
    profits[key_word] = joined.iloc[-1]['Total Profit'] # max(joined.iloc[-1]['Total Profit'], -1 * joined.iloc[-1]['Total Profit'])
    
    if profits[key_word] == joined.iloc[-1]['Total Profit']: # negative sentiment word, naive classification
        word_types['Negative'].append(key_word)
    else:
        word_types['Positive'].append(key_word)
        
    
    return joined

In [84]:
word_trends = {}    #dictionary with key=word, value=DataFrame with Google trend and pct change 
for word in words:
    word_trends[word] = get_trend(word, 2014, 2014, 1, 6, profits, word_types)

hedge:2020-01-01 2020-01-31
hedge:2020-02-01 2020-02-29
hedge:2020-03-01 2020-03-31
hedge:2020-04-01 2020-04-30
hedge:2020-05-01 2020-05-31
hedge:2020-06-01 2020-06-30
[*********************100%***********************]  1 of 1 completed
dividend:2020-01-01 2020-01-31
dividend:2020-02-01 2020-02-29
dividend:2020-03-01 2020-03-31
dividend:2020-04-01 2020-04-30
dividend:2020-05-01 2020-05-31
dividend:2020-06-01 2020-06-30
[*********************100%***********************]  1 of 1 completed
earnings:2020-01-01 2020-01-31
earnings:2020-02-01 2020-02-29
earnings:2020-03-01 2020-03-31
earnings:2020-04-01 2020-04-30
earnings:2020-05-01 2020-05-31
earnings:2020-06-01 2020-06-30
[*********************100%***********************]  1 of 1 completed
inflation:2020-01-01 2020-01-31
inflation:2020-02-01 2020-02-29
inflation:2020-03-01 2020-03-31
inflation:2020-04-01 2020-04-30
inflation:2020-05-01 2020-05-31
inflation:2020-06-01 2020-06-30
[*********************100%***********************]  1 of 1 co

In [85]:
word_trends['debt']

,Date,debt,change,Adj Close,Diff,Recent Profit,Total Profit
0,2020-01-02,60.04,1.777058,28868.800781,NaN,0.000000,0.000000
1,2020-01-03,57.72,-0.038641,28634.880859,-233.919922,68.500000,68.500000
2,2020-01-06,61.62,0.715957,28703.380859,68.500000,-119.701172,-51.201172
3,2020-01-07,79.12,0.283999,28583.679688,-119.701172,161.410156,110.208984
4,2020-01-08,58.32,-0.262892,28745.089844,161.410156,211.810547,322.019531
...,...,...,...,...,...,...,...
98,2020-05-22,54.40,-0.006937,24465.160156,-8.958984,529.949219,-3639.771484
99,2020-05-26,46.50,0.366843,24995.109375,529.949219,553.160156,-3086.611328
100,2020-05-27,53.72,0.155269,25548.269531,553.160156,-147.628906,-3234.240234
101,2020-05-28,50.70,-0.056217,25400.640625,-147.628906,-17.531250,-3251.771484


In [51]:
# old data
profits

{'hedge': 5132.955078125,
 'dividend': -444.826171875,
 'earnings': -8195.619140625,
 'inflation': -8356.125,
 'markets': 13238.6796875,
 'debt': -3884.408203125}

In [60]:
# new data
profits

{'hedge': 9979.392578125,
 'dividend': -269.267578125,
 'earnings': -7950.978515625,
 'inflation': -8080.42578125,
 'markets': 13029.96875,
 'debt': -4449.990234375}

In [52]:
word_types

defaultdict(set,
            {'Negative': {'debt',
              'dividend',
              'earnings',
              'hedge',
              'inflation',
              'markets'},
             'Positive': {'debt', 'dividend', 'earnings', 'inflation'}})

In [ ]:
# new data

In [43]:
# old data
word_trends["debt"]

,Date,debt,change,Adj Close,Diff,Recent Profit,Total Profit
0,2020-01-02,58.46,1.764066,28868.800781,NaN,0.000000,0.000000
1,2020-01-03,57.72,-0.012658,28634.880859,-233.919922,-68.500000,-68.500000
2,2020-01-06,63.99,0.665105,28703.380859,68.500000,119.701172,51.201172
3,2020-01-07,76.36,0.193311,28583.679688,-119.701172,161.410156,212.611328
4,2020-01-08,63.18,-0.172603,28745.089844,161.410156,-211.810547,0.800781
...,...,...,...,...,...,...,...
98,2020-05-22,54.40,-0.089692,24465.160156,-8.958984,-529.949219,-3124.169922
99,2020-05-26,44.25,0.211002,24995.109375,529.949219,-553.160156,-3677.330078
100,2020-05-27,52.14,0.178305,25548.269531,553.160156,0.000000,-3677.330078
101,2020-05-28,48.36,-0.072497,25400.640625,-147.628906,17.531250,-3659.798828


## Old Stuff

In [10]:
fortune_500 = pd.read_csv("Fortune 500 2017 - Fortune 500.csv")
fortune_500.head()

,Rank,Title,Website,Employees,Sector,Industry,Hqlocation,Hqaddr,Hqcity,Hqstate,Hqzip,Hqtel,Ceo,Ceo-title,Address,Ticker,Fullname,Revenues,Revchange,Profits,Prftchange,Assets,Totshequity
0,1,Walmart,http://www.walmart.com,2300000,Retailing,General Merchandisers,"Bentonville, AR",702 S.W. Eighth St.,Bentonville,AR,72716,479-273-4000,C. Douglas McMillon,"President, Chief Executive Officer & Director","702 S.W. Eighth St., Bentonville, AR 72716",WMT,"Wal-Mart Stores, Inc.",485873,0.8,13643.0,-7.2,198825,77798.0
1,2,Berkshire Hathaway,http://www.berkshirehathaway.com,367700,Financials,Insurance: Property and Casualty (Stock),"Omaha, NE",3555 Farnam St.,Omaha,NE,68131,402-346-1400,Warren E. Buffett,Chairman & Chief Executive Officer,"3555 Farnam St., Omaha, NE 68131",BRKA,Berkshire Hathaway Inc.,223604,6.1,24074.0,0,620854,283001.0
2,3,Apple,http://www.apple.com,116000,Technology,"Computers, Office Equipment","Cupertino, CA",1 Infinite Loop,Cupertino,CA,95014,408-996-1010,Timothy D. Cook,Chief Executive Officer & Director,"1 Infinite Loop, Cupertino, CA 95014",AAPL,"Apple, Inc.",215639,-7.7,45687.0,-14.4,321686,128249.0
3,4,Exxon Mobil,http://www.exxonmobil.com,72700,Energy,Petroleum Refining,"Irving, TX",5959 Las Colinas Blvd.,Irving,TX,75039,972-444-1000,Darren W. Woods,Chairman & Chief Executive Officer,"5959 Las Colinas Blvd., Irving, TX 75039",XOM,Exxon Mobil Corporation,205004,-16.7,7840.0,-51.5,330314,167325.0
4,5,McKesson,http://www.mckesson.com,68000,Wholesalers,Wholesalers: Health Care,"San Francisco, CA",1 Post St.,San Francisco,CA,94104,415-983-8300,John H. Hammergren,"Chairman, President & Chief Executive Officer","1 Post St., San Francisco, CA 94104",MCK,McKesson Corporation,192487,6.2,2258.0,53,56563,8924.0


In [11]:
def get_name(symbol):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(symbol)

    result = requests.get(url).json()

    for x in result['ResultSet']['Result']:
        if x['symbol'] == symbol:
            return x['name']

In [12]:
def get_data(ticker: str):
    print("Stock data for: \"" + str(ticker) + "\"")
    data = pd.DataFrame(yf.download(ticker, start = '2020-1-1', end = "2020-6-1", interval = "1d")["Adj Close"])
    name = fortune_500[fortune_500["Ticker"] == ticker][["Title"]].iloc[0][0]
    print("Search trends for: \"" + str(name) + "\"")
    trend = dailydata.get_daily_data(name, 2020, 1, 2020, 6, wait_time = 0)[name]
    data.index.name = "date"
    data = data.join(trend, how = "inner")
    return data

In [13]:
df = get_data("TSLA")
df.head(50)

Stock data for: "TSLA"
[*********************100%***********************]  1 of 1 completed
Search trends for: "Tesla"
Tesla:2020-01-01 2020-01-31
Tesla:2020-02-01 2020-02-29
Tesla:2020-03-01 2020-03-31
Tesla:2020-04-01 2020-04-30
Tesla:2020-05-01 2020-05-31
Tesla:2020-06-01 2020-06-30


,Adj Close,Tesla
date,,
2020-01-02,86.052002,21.24
2020-01-03,88.601997,24.79
2020-01-06,90.307999,15.50
2020-01-07,93.811996,22.94
2020-01-08,98.428001,25.35
2020-01-09,96.267998,23.94
2020-01-10,95.629997,18.81
2020-01-13,104.972000,23.40
2020-01-14,107.584000,26.52
